Imports

In [69]:
from __future__ import print_function
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.autograd import Variable
from typing import Tuple
from torch.utils.data import Dataset,DataLoader
# from sklearn.metrics import confusion_matrix, top_k_accuracy_score
import torchvision                                                       
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torchvision.transforms import ToTensor
from numpy import random as rd
import torch
import torch.nn as nn
import torch.optim as optim
import gc

Dataloader Block

In [70]:
# train data
trainData = datasets.FashionMNIST(root="./",
                                  train=True,
                                  transform=ToTensor(),
                                  download=True
                                  )
trainLoad = DataLoader(trainData, 
                       batch_size=60, 
                       shuffle=True, 
                       drop_last=False
                       )
# test data
testData = datasets.FashionMNIST(root="./",
                                  train=False,
                                  transform=ToTensor(),
                                  download=True
                                  )
testLoad = DataLoader(testData, 
                     batch_size=60, 
                     shuffle=True, 
                     drop_last=False
                     )

Residual Block

In [71]:
class ResidualBlock(nn.Module):
    expansion = 4
    
    def __init__(self, in_channels, out_channels, stride = 1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels*2, kernel_size=1, stride=stride, padding=0),
            nn.BatchNorm2d(out_channels*2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels*2, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(out_channels*2)
        )

        # downsample for forwarding
        short = []
        if stride != 1 or in_channels != out_channels * self.expansion:
            short.append(nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, stride=stride, padding=0))
            short.append(nn.BatchNorm2d(out_channels * self.expansion))
        self.short = nn.Sequential(*short)
        self.relu = nn.ReLU()
        
    def forward(self, residual):
        # out = torch.cat([self.conv1(residual), self.conv2(residual)], 1)
        # residual = self.short(residual)
        # out = self.relu(out + residual)
        
        out = self.conv1(residual)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(residual)
        out += residual
        out = self.relu(out)
        return out

Renset Block

In [72]:
# class for ResNet model that extend from nn.Module
class Resnet(nn.Module):
    # initialize the resnet model with inputted block type, list of blockNum 
    def __init__(self, block, blockList):
        super(Resnet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels= 1, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(3, 2, 1)
        )
        
        # 3, 3, 3, 3 blocks
        # self.block0 = self._make_layer(block,   inChannels=64,  outChannels=64, blocksNum=blockList[0], stride=1)
        # self.block1 = self._make_layer(block,  inChannels=256, outChannels=128, blocksNum=blockList[1], stride=1)
        # self.block2 = self._make_layer(block,  inChannels=512, outChannels=256, blocksNum=blockList[2], stride=1)
        # self.block3 = self._make_layer(block, inChannels=1024, outChannels=512, blocksNum=blockList[3], stride=2)
        
        self.block0 = self._make_layer(block,   inChannels=128,  outChannels=64, blocksNum=blockList[0], stride=1)
        self.block1 = self._make_layer(block,  inChannels=256, outChannels=128, blocksNum=blockList[1], stride=1)
        self.block2 = self._make_layer(block,  inChannels=512, outChannels=256, blocksNum=blockList[2], stride=1)
        self.block3 = self._make_layer(block, inChannels=1024, outChannels=512, blocksNum=blockList[3], stride=2)
        
        # apply 2D adaptive average pooling from 1 input to 1 plane
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        # flatten the data into 1 dimension
        self.flatten = nn.Flatten()
        # apply dropout to output with 60% percent chance
        self.drop = nn.Dropout(0.6)
        # connect 2048 input nodes into 10 output nodes
        self.fc = nn.Linear(2048, 10)

    # helper function that adds layer by layer along with the res block
    def _make_layer(self, block, inChannels, outChannels, blocksNum, stride):
        layers = []
        layers.append(block(inChannels, outChannels, stride))

        inChannels = outChannels * block.expansion
        
        for _ in range(1, blocksNum):
            layers.append(block(inChannels, outChannels, 1))

        return nn.Sequential(*layers)
    
    # forward function 
    def forward(self, x):
        x = self.conv1(x)
        x = self.block0(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.avgpool(x)
        x = self.drop(x)
        x = self.fc(x)
        return x

Hyperparameter block

In [73]:
# number of epoch
epochNum=20
# learning rate
learningRate = 0.01
# weight decay
weightDecayRate = 0.001
# momentum
momentumAmount = 0.9
# setting up the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# setting up the model
model = Resnet(ResidualBlock, [3, 4, 6, 3]).to(device)
# loss
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate, weight_decay=weightDecayRate)

Training

In [74]:
total_step = len(trainLoad)

for epoch in range(epochNum):
    for i, (images, labels) in enumerate(trainLoad):
        # move tensor to device
        images = images.to(device)
        labels = labels.to(device)
        
        # forward the output and calculate loss
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward the output and perform optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # deallocation
        del images, labels, outputs
        torch.cuda.empty_cache()
        gc.collect()
        
#    print ('Epoch [{}/{}], Loss: {:.4f}' 
#                   .format(epoch+1, epochNum, loss.item()))
    

RuntimeError: Given groups=1, weight of size [64, 128, 1, 1], expected input[60, 64, 14, 14] to have 128 channels, but got 64 channels instead

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.cpu(), y.cpu()
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss ,current = loss.item(), batch * len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn, Train = False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.cuda(), y.cuda()
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    if Train:
        print(f"Train Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    else:
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

def train_loop(model, epochs):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(trainLoad, model, loss_fn, optimizer)
        test(trainLoad, model, loss_fn, Train = True)
        test(trainLoad, model, loss_fn)
    print("Done!")

In [ ]:
resnet_model = Resnet(ResidualBlock, [3, 4, 6, 3])
train_loop(resnet_model.cpu(), 20)

Testing

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testLoad:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs
    
    print('Accuracy of the network on the {} validation images: {} %'.format(10000, 100 * correct / total))